# Text Cleaning

In this notebook we perform essential text data cleaning.

Implemented methods are very general and can be useful for various datasets. If neccesary, you can implement some additional cleaning methods and  add it easially to final cleaning method.

### Imports

In [0]:

import sys, os
import nltk
import numpy as np
import pandas as pd
import re
import time

from google.colab import drive
from nltk.corpus import stopwords
from sklearn.externals import joblib

%reload_ext autoreload
%autoreload 2


pd.set_option('display.max_colwidth', -1)

### Colab setup

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cd ~/..



In [23]:
os.getcwd()

'/'

**Notice:** There is an issue related to whitespace in "My Drive" when trying to invoke path with it. As for now google colab does not allow to rename e.g. to "MyDrive". One walkaround is to create a symbolic link to omit problem with whitespace within "My Drive". 


In [0]:
# Create a symbolic link to omit issues with whitespace in "My Drive"
!ln -s ~/../content/gdrive/"My Drive"/ MyDrive


In [0]:
PROJECT_HOME_PATH = os.path.join('MyDrive', 'NmtPolishLanguage')
DATA_PATH = os.path.join(PROJECT_HOME_PATH, 'DATA')

In [25]:
os.path.exists(PROJECT_HOME_PATH)

True

In [26]:
os.getcwd()

'/'

In [27]:
ls

bin/      datalab/  home/   lib64/  MyDrive@  root/  srv/    tmp/    var/
boot/     dev/      lib/    media/  opt/      run/   swift/  tools/
content/  etc/      lib32/  mnt/    proc/     sbin/  sys/    usr/


### Load dataset

In [0]:
df_data = pd.read_csv(os.path.join(DATA_PATH, 'hate_speech_text.txt'), sep="\n", header=None, names=["text"])
target = pd.read_csv(os.path.join(DATA_PATH, 'hate_speech_tags.txt'), sep="\n", header=None, names=["target"])

In [0]:
df_data = pd.concat([df_data, target], axis=1)

In [30]:
df_data.head()

,text,target
0,"Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.",0
1,@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe,0
2,"@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.",0
3,@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem",0


### Look at examples

In [0]:
text = df_data['text']

In [32]:
#Display some examples
for row in text[100:110]:
  print(row)

@anonymized_account Równie dobrze można gola stracić :)
@anonymized_account Chorwaci przerżną w końcówce dogrywki, wiadomo.
@anonymized_account Nasze dzieci będą zdegustwani, był taki PiS, taki @anonymized_account , historia populizmu i głupoty w latach 2015-2019, margines
@anonymized_account O, myślałam, że nikt już nie używa \"mnie\" tylko \"mi\".
@anonymized_account @anonymized_account @anonymized_account Na nowym stadionie West Hamu też to widziałem.
@anonymized_account @anonymized_account zawodnik w sezonie może grać w barwach dwóch klubów więc jeszcze nic straconego :)
ciekawe czy faktycznie jest taki zakład bo jeśli tak to nieźle XD
@anonymized_account Hehe, do gazu, hehe jeszcze jakieś emotikony wrzuć
Qrwa!! I niech mi nikt nie mówi, że czary to jakiś zabobon!! Chłopaki mają sztywne nogi!
Od 01.10.2018 każde sprawozdanie finansowe w formie elektronicznej, zgodne ze strukturą jpk


### Data cleaning

In [0]:
# Load Stopwords
stopwords_from_file = True

In [0]:
if stopwords_from_file:
  with open(os.path.join(PROJECT_HOME_PATH, 'polish_stopwords.txt'), encoding="utf-8", newline='\n') as f:
    stopwords = f.readlines()
    
  stopwords_list = [x[:-1] for x in stopwords]

else:
  nltk.download('stopwords')
  stopwords_list = list(set(stopwords.words('english')))

In [35]:
stopwords_list[:10]

['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ']

In [0]:
def compareChanges(text, func):
    """Function to compare changes due to applied function.

    Args:
        text, pd.Series: string to modify.
        func, def: function which returns modified text.

    Returns:
        text_mod, pd.Dataframe: result df with columns: TextBefore, TextAfter, Changed.
    """

    text_mod = pd.DataFrame(columns=['TextBefore', 'TextAfter', 'Changed'])
    text_mod['TextBefore'] = text.copy()

    for index, row in text_mod.iterrows():
        row['TextAfter'] = func(row['TextBefore'])

    text_mod['Changed'] = np.where(text_mod['TextBefore']==text_mod['TextAfter'], 0, 1)

    return text_mod
  
  
def removeNumbers(text):
    text = re.sub('[0-9]', '', text)
    return text

  
def removeAllOtherThanLetters(text):
    text = re.sub('[^a-zA-Z ]', '', text)
    return text
  
  
def makeLowerCase(text):
    return text.lower()
  
  
def removeSpecial(text):
    puncts = ['&']
    
    
    for punct in puncts:
        text = text.replace(punct, '')
        
    return text
  
  
def removeWhitespace(text):
    text = " ".join(text.split())
    return text
  

def removeStopwords(text, stopwords_list):
    final_tokens = []
    tokens = text.split()
    
    for word in tokens:
        if (word not in stopwords_list):
            final_tokens.append(word)
            
    text = " ".join(final_tokens)
  
    return text
  
  
def removeSelectedWords(text):
    final_tokens = []
    tokens = text.split()
    selected_words = ['anonymizedaccount']
    for word in tokens:
        if (word not in selected_words):
            final_tokens.append(word)
            
    text = " ".join(final_tokens)
    
    return text
  
  
def removeShortWords(text):
    final_tokens = []
    tokens = text.split()
    
    for word in tokens:
        if len(word) > 1:
            final_tokens.append(word)
            
    text = " ".join(final_tokens)
    
    return text
    
    
def replacePolishLetters(text):
  
  
    patterns = [ (r'ł', 'l'), 
                 (r'ó', 'o'),
                 (r'ą', 'a'),
                 (r'ć', 'c'),
                 (r'ę', 'e'),
                 (r'ń', 'n'),
                 (r'ś', 's'),
                 (r'ź', 'z'),
                 (r'ż', 'z')]
  
    for (pattern, repl) in patterns:
        text = re.sub(pattern, repl, text)
    return text
            
            
def cleanData(text):
  
    final_text = []
    
    for w in text:
        w = replacePolishLetters(w)
        w = removeAllOtherThanLetters(w)
        w = makeLowerCase(w)
        w = removeStopwords(w, stopwords_list)
        w = removeSelectedWords(w)
        w = removeShortWords(w)
        w = removeWhitespace(w)    

        
        final_text.append(w)
        
    return pd.Series(final_text, index=text.index)
  
  
def cleanDataToDF(text):
  
    text_mod = pd.DataFrame(columns=['TextBefore', 'TextAfter'])
    text_mod['TextBefore'] = text.copy()
    
    for index, row in text_mod.iterrows():
        row['TextAfter'] = replacePolishLetters(row['TextBefore'])
        row['TextAfter'] = removeAllOtherThanLetters(row['TextBefore'])
        row['TextAfter'] = makeLowerCase(row['TextBefore'])
        row['TextAfter'] = removeStopwords(row['TextBefore'], stopwords_list)  
        row['TextAfter'] = removeSelectedWords(row['TextBefore'])  
        row['TextAfter'] = removeShortWords(row['TextBefore'])        
        row['TextAfter'] = removeWhitespace(row['TextBefore'])    

    return text_mod
  
  

#### This method is to verify how particular cleaning method works:

In [37]:
text_mod = compareChanges(text[:20], replacePolishLetters)
text_mod.head()

,TextBefore,TextAfter,Changed
0,"Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.","Dla mnie faworytem do tytulu bedzie Cracovia. Zobaczymy, czy typ sie sprawdzi.",1
1,@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe,@anonymized_account @anonymized_account Brawo ty Daria kibic ma byc na dobre i zle,1
2,"@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.","@anonymized_account @anonymized_account Super, polski premier sklada kwiaty na grobach kolaborantow. Ale doczekalismy czasow.",1
3,@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.,@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.,0
4,"Odrzut natychmiastowy, kwaśna mina, mam problem","Odrzut natychmiastowy, kwasna mina, mam problem",1


### Run cleaning

In [0]:
# We fill null values with string 'empty string'
text = text.fillna('puste pole')


# Run data cleaning
text_mod = cleanData(text)

In [39]:
text_mod[100:110]

100    rownie dobrze mozna gola stracic                                          
101    chorwaci przerzna koncowce dogrywki wiadomo                               
102    dzieci zdegustwani pis historia populizmu glupoty latach margines         
103    myslalam nikt nie uzywa                                                   
104    nowym stadionie west hamu tez widzialem                                   
105    zawodnik sezonie moze grac barwach dwoch klubow wiec straconego           
106    ciekawe faktycznie zaklad niezle xd                                       
107    hehe do gazu hehe jakies emotikony wrzuc                                  
108    qrwa nikt nie mowi czary zabobon chlopaki maja sztywne nogi               
109    od kazde sprawozdanie finansowe formie elektronicznej zgodne struktura jpk
dtype: object

In [0]:
# If we did not ommit any record, then add columns with cleaned text

if len(text) == len(text_mod):
        df_data['text_mod'] = text_mod

In [0]:
# Rearrange columns order
df_data = df_data[['text', 'text_mod', 'target']]


# If any cleaned text resulted with as 0 len string '', we drop it
df_data = df_data.dropna()
df_data = df_data.drop(df_data[df_data.text_mod == ''].index)

In [42]:
df_data.tail(5)

,text,text_mod,target
10036,@anonymized_account Ty zagrasz? Nie wiedziałem 😉,zagrasz nie wiedzialem,0
10037,"@anonymized_account @anonymized_account A VAR nie miał poprawić jakości sędziowania, tylko efekt końcowy - mniej wypaczonych wyników, mniej skandali.",var nie mial poprawic jakosci sedziowania efekt koncowy mniej wypaczonych wynikow mniej skandali,0
10038,"@anonymized_account @anonymized_account Szanowany, bo kolega ładnie go pożegnał ?",szanowany kolega ladnie pozegnal,0
10039,@anonymized_account @anonymized_account @anonymized_account A kto inny ma się bić? Każdy zwyciezca ligi wojewódzkiej gra w barazach.,bic zwyciezca ligi wojewodzkiej gra barazach,0
10040,@anonymized_account A wróżbita Maciej mówi że zrozumiemy,wrozbita maciej mowi zrozumiemy,0


#### Save cleaned data

In [43]:
joblib.dump(df_data, os.path.join(DATA_PATH, 'interim', 'hate_speach_mod.dat'))

['MyDrive/NmtPolishLanguage/DATA/interim/hate_speach_mod.dat']